In [2]:
import pymc3 as pm
from prior_comparison_tool.main import create_app
import pandas as pd
from prior_comparison_tool.model import model
import numpy as np

# ********** model taken from statistical rethinking - McElreath 2015 *************

def model_method(data, **prior_params):
    weight_m = np.vstack((data.weight_std, data.weight_std**2, data.weight_std**3))
    with pm.Model() as model:
        alpha = pm.Normal('alpha', mu=prior_params['alpha mu'], sd=prior_params['alpha sd'])
        beta = pm.Normal('beta', mu=prior_params['beta mu'], sd=prior_params['beta sd'], shape=3)
        sigma = pm.Uniform('sigma', lower=prior_params['sigma lower'], upper=prior_params['sigma upper'])
        mu = pm.Deterministic('mu', alpha + pm.math.dot(beta, weight_m))
        height = pm.Normal('height', mu=mu, sd=sigma, observed=data.height)
    return model

d = pd.read_csv("Howell1.csv", sep=";", header=0)
d = d[d.age >= 18]
d["weight_std"] = (d.weight - d.weight.mean()) / d.weight.std()
d["weight_std2"] = d.weight_std**2


params= {
    'alpha mu':178,
    'alpha sd':100,
    'beta mu':0,
    'beta sd':10,
    'sigma lower':0,
    'sigma upper':50,
}

data = d

m = model_method(data, **params)

In [3]:
m

In [4]:
import arviz as az

with m:
    prior = pm.sample_prior_predictive()
    trace = pm.sample()
    posterior = pm.sample_posterior_predictive(trace)
    pm_data = az.from_pymc3(
        prior=prior,
        posterior_predictive=posterior,
        trace=trace,
    )

In [7]:
az.plot_pair(pm_data, backend='bokeh')

array([[Figure(id='1002', ...), None, None, None, None, None, None, None],
       [Figure(id='1038', ...), Figure(id='1074', ...), None, None, None,
        None, None, None],
       [Figure(id='1110', ...), Figure(id='1146', ...),
        Figure(id='1182', ...), None, None, None, None, None],
       [Figure(id='1218', ...), Figure(id='1254', ...),
        Figure(id='1290', ...), Figure(id='1326', ...), None, None, None,
        None],
       [Figure(id='1362', ...), Figure(id='1398', ...),
        Figure(id='1434', ...), Figure(id='1470', ...),
        Figure(id='1506', ...), None, None, None],
       [Figure(id='1542', ...), Figure(id='1578', ...),
        Figure(id='1614', ...), Figure(id='1650', ...),
        Figure(id='1686', ...), Figure(id='1722', ...), None, None],
       [Figure(id='1758', ...), Figure(id='1794', ...),
        Figure(id='1830', ...), Figure(id='1866', ...),
        Figure(id='1902', ...), Figure(id='1938', ...),
        Figure(id='1974', ...), None],
       [F